# 第8章 字典和集合
## 8.1 数据存储,检索和字典
计算机的基本功能是存储和处理数据,首先是存储数据,处理也需要先找到被处理的数据,或成为**访问数据**.所以,数据的存储和访问是计算机最基本的功能.
### 8.1.1 数据存储和检索
数据访问的基本方式是基于存储位置.

找到数据的存储位置的操作称为__检索__,类似查字典.
#### 概述
数据检索牵涉到两个方面,一方面是已存储的数据集合,另一方面是用户检索时提供的信息.具体检索可以是确定特定数据是否存在于数据集中,相当于集合成员判断;也可以是希望找到与所提供信息相关的数据,类似与在字典里查询词语的解释.在后一方式中,检索时提供的信息被看做`检索码`或`关键码`(key).这种关键码也常作为数据的一部分,存储在数据集里,这就是**基于关键码的数据存储和检索**,是本章讨论的主题.

作为检索基础的关键码,通常是数据项的某种(可能具有唯一性的)特征,可以是数据内容的一个组成部分,也可以是专门为数据检索建立的标签.后一种情况实际中也很常见.以学校的学生记录为例,原本只需要记录与学生相关的信息.而为每个学生指定一个学号,只是为了检索方便.这一类关键码通常是数据的唯一`标识`.

基本假设(抽象的):需要存储的数据元素由两个部分组成,其中一部分是与检索相关的关键码,另一部分是与之关联的数据.

`字典`就是支持基于关键码的数据存储与检索的数据结构.也被称为`查找表`,`映射`或者`关联表`等.字典实现数据的存储和检索,而需要存储和检索的信息和环境有许多具体情况,因此可能要求不同的实现技术.

在字典的使用中,最重要也是使用最频繁的操作就是检索(search,也称查找).
#### 字典操作和效率
实际中使用的字典可以分为两类:
- 静态字典:在建立之后,这种字典内容和结构都不再变化,主要操作只有检索.对这种字典,需要考虑创建的代价,但最重要的是检索效率.
- 动态字典:在初始创建之后,这种字典的内容(和结构)将一直处于动态变动之中.对这种字典,除了检索之外,最重要的基本操作还包括数据项的插入和删除等.

对于动态字典,还有一个问题也必须重视:这里的插入和删除操作可能导致字典结构的变化.良好设计的字典应该保证其性能不会随着操作的进行而逐渐恶化.

在字典中检索,最终将得到一个结果,或是检索成功而且得到了所需的数据;或是确认了要找的数据不存在,此时可能返回某种特殊标志.有关检索效率的评价标准,通常考虑的是在一次完整检索过程中比较关键码的平均次数,通常称为`平均检索长度`(Average Search Length,ASL),其定义是(其中n为字典中的数据项数):$$ASL=\sum_{i=0}^{n-1}{p_i\cdot{c_i}}$$其中$c_i$和$p_i$分别为第i项数据元素的检索长度和检索概率.上述定义中只考虑了被检索关键码在字典中存在的情况(正确的检索算法保证存在的关键码一定能找到),在很多情况中,还需要考虑字典中不存在被检索关键码的情况.\

#### 字典和索引
实际上,字典是两种功能的统一:
- 作为一种数据存储结构,支持在字典里存储一批数据项.
- 提供支持数据检索的功能,设法维护从关键码找到相关数据的联系信息.

后一功能称为`索引`,其存在的目的就是为检索服务.

做基于关键码的检索,就是要实现从关键码到数据存储位置的映射,而这种映射也就是索引.但是,索引结构本身并不存储数据,只提供(基于关键码的)检索功能,因此它只作为字典的附属结构,不可能独立存在.
### 8.1.2 字典的实现问题
#### 字典抽象数据类型
|ADT Dict:|# 字典抽象数据类型|
|---|---|
|Dict(self)|# 字典构造函数,创建一个新字典|
|is_empty(self)|# 判断self是否为一个空字典|
|num(self)|# 获知字典中的元素个数|
|search(self,key)|# 检索字典里key的关联数据|
|insert(self,key,value)|# 将关联(key,value)加入字典|
|delete(self,key)|# 删除字典中关键码为key的元素|
|values(self)|# 支持以迭代方式取得字典里保存的各项关联中的value|
|entries(self)|# 支持以迭代方式获得字典的关联中的逐堆key和value二元组|

注意,各种字典都不应该允许修改字典关联中的关键码,因为关键码用于确定其所在项在字典里存储的位置,以支持高效检索.如果允许修改关键码,就可能破坏字典数据结构的完整性,导致后续检索操作失败.
#### 字典元素:关联
在支持基于关键码的存储与检索的字典里,一个数据项可以划分为两个部分:其一是与检索有关的关键码部分,另外就是与检索无关的其他数据部分.下面将始终把字典里的数据项简单地分为两部分,一部分是与检索有关的关键码,另一部分称为值,与检索和其他操作(例如插入/删除)的实现方式无关,但在实际应用中却可能非常重要.这样,一个数据项就是一种`二元组`,下面称之为`关联`.


In [1]:
class Assoc:
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def __lt__(self, other):
        return self.key < other.key
    def __le__(self, other):
        return self.key < other.key or self.key == other.key
    def __str__(self):      # 定义字符串表示形式便于输出和交互
        return "Assoc({0},{1})".format(self.key, self.value)

#### 字典的实现
字典实现技术:线性表,连续表,散列表
## 8.2 字典线性表实现
线性表里可以存储信息,因此可以作为字典的实现基础
### 8.2.1 基本实现
基于线性表的字典实现,优点和缺点都非常明显:
- 数据结构和算法都很简单,检索,删除等操作中只需要比较关键码相同(或不同),适用于任何关键码类型(例如,并不要求关键码集合存在某种顺序关系)
- 平均检索效率第(线性时间),表长度n较大时,检索很耗时.
- 删除操作的效率也比较地,因此不太适合频繁变动的字典.

另外,在字典的动态变化中,这种字典的各种操作的效率不变.但这并不时什么优点,因为它们都已经时效率最低的操作了.
### 8.2.2 有序线性表和二分法检索
具有合理内部结构的字典上有可能实现高效检索.

如果关键码取自一个`有序集合`(存在某种内在的序,例如整数的小于等于关系,字符串的字典序等),就可以按照关键码大小的顺序排列字典里的项(从小到大或从大到小).在数据项按序排列时,可以采用`二分法`实现快速检索.

In [2]:
def bisearch(lst, key):
    low, high = 0, len(lst)-1
    while low <= high:        # 范围内还要元素
        mid = low + (high - low) // 2
        if key == lst[mid].key:
            return lst[mid].value
        if key < lst[mid].key:
            high = mid - 1    # 在低半区继续
        else:
            low = mid + 1     # 在高半区继续

#### 二分法检索实例
用二叉树表示对表中所有元素的检索过程,这样的树被称为`二分法检索过程的判定树`.树中结点所标的数时数据项的关键码.

采用有序的顺序表和二分检索法:
- 主要优点时检索速度块,$O(log n)$.
- 插入和删除时需要维护数据项的顺序,因此都是O(n)操作(虽然检索插入/删除位置可以用二分法,为O(log n)时间,但完成操作需要移动元素,为O(n)时间
- 二分技术只能用于关键码可以排序,数据项按关键码排序的字典,而且只适用于顺序存储结构,需要连续的存储块,不适合实现很大的动态字典
### 8.2.3 字典线性表总结
虽然也可以考虑基于单链表或双链表技术实现字典,但通过分析可知:
- 如果字典里的数据项任意排列,插入时可以简单地在表头插入,是O(1)操作;检索和删除都要扫描整个表,是O(n)操作.
- 如果表中的数据项按关键码升序或者降序排列,插入需要检索正确位置,为O(n)操作;检索和删除同样需要顺序扫描检查,平均检查半个表,为O(n)操作.

由上述分析可知:采用链接表实现字典没有任何明显的优势,而且无法利用关键码排序的价值.
#### 问题和思考
采用线性表技术实现字典,只能适合一部分简单需求,如字典的规模比较小,而且不常出现动态操作的情况.但在实际中,各种应用经常需要存储和检索很大的数据集合,字典中的数据内容不断动态变化的情况也很常见.采用简单的顺序表或链接表实现字典,顺序检索的效率太低,在效率上常常不能满足实际应用的需要.采用排序的顺序表和二分检索能大大提高检索速度,但仍有两大问题:
- 不能很好地支持数据地变化(数据插入和删除地效率低)
- 必须采用连续方式存储整个数据集合.如果数据集很大,连续存储方式就很难接受了.

如果要支持存储很大地,经常变动的数据集合,而且希望高效检索,必须考虑其他组织方式.人们为此开发了另外一些结构,主要分为两类:
- 基于散列(hash)思想的`散列表(也称为`哈希表`)
- 基于各种树形结构的数据存储和检索技术(利用树结构的特性,即在不大的深度范围内可以容纳巨大数量的结点)
## 8.3 散列和散列表
在计算机科学技术领域使用非常广泛的散列技术及其在字典方面的应用,即所谓的`散列表`(hash table).
### 8.3.1 散列的思想和应用
在什么情况下基于关键码能最快找到所需的数据?

如果数据项连续存储,而关键码就是存储数据的地址(或下标)!显然,在这种情况下,只需O(1)时间就能得到所需要的数据了.

但是,一般而言,字典所用的关键码可能不是整数,由此不能作为下标.另一方面,即使关键码是整数,也可能因为取值范围太大而不适合作为下标.

散列表的基本思想:如果一种关键码不能或者不适合作为数据存储的下标,可以考虑通过一个变换(一个计算)把它们映射到一种下标.这样做,就把基于关键码的检索转变为基于整数下标的直接元素访问,有可能得到一种高效的字典实现技术.

以散列表的思想实现字典,具体方法是:
 1. 选定一个整数的下标范围(通常以0或1开始),建立一个包括相应元素位置范围的顺序表.
 2. 选定一个从实际关键码集合到上述下标范围的适当映射h:
     - 在需要存入关键码为key的数据时,将其存入表中第h(key)个位置.
     - 遇到以key为关键码检索数据时,直接去找表中第h(key)个位置的元素.
     
这个h称为`散列函数`,也常被称为哈希(hash)函数或杂凑函数,它就是从可能的关键码集合到一个整数区间(下标区间)的映射
#### 散列思想在信息领域的应用
所谓`散列`,一般而言,就是以某种精心设计的方式,从一段可能很长的数据生成一段很短(经常为固定长度)的信息串,例如简单的整数或字符串,最终都是二进制串,而后利用这种二进制串做某种有用的事情.散列技术在计算机和信息技术领域非常有价值,应用极广,可能用在各种数据处理,存储,检索工作中.例如:
- 文件的完整性检查:
- 互联网技术中到处都是用和依靠散列函数,用于网页传输中的各种安全性和正确性检查,包括各种网络认证和检查,各种网络协议
- 计算机安全领域中大量使用散列技术,例如在各种安全协议中.

将散列技术应用于数据的存储和检索,就得到了散列表.实现一个散列表,不但需要选择合适的关键码映射(散列函数),还要考虑由于采用映射方式确定存储和检索位置而带来的各种问题.
#### 散列技术:设计和性质
下标集合通常都远远小于关键码集合的规模.在通常情况下,散列函数h时从一个大集合到一个小集合的映射.在这种情况下,它显然不可能是单射,必然会出现多个不同的关键码被h对应到同一个下标位置的情况.如果实际中出现了这种情况,人们就说这里出现了`冲突`(或者`碰撞`),此时也称$key_1$和$key_2$为h下的`同义词`.根据上面分析,冲突是散列表使用中必然会出现的情况,因此,在考虑散列表的实现时,必须考虑如何解决冲突的问题.

对于一个规模固定的散列表,一般而言,表中的元素越多,出现冲突的可能性也就越大.这里的一个重要概念是`负载因子`.这是一种很有用的度量值,是考察散列表在运行中的性质的一个重要参数,其定义是:$$负载因子\alpha = \frac{散列表中当时的实际数据项数}{散列表的基本存储区能容纳的元素个数}$$如果数据项直接保存在基本存储区里,那么将总会有$\alpha \le 1$.无论如何,负载因子的大小与冲突出现的可能性密切相关:负载因子越大,出现冲突的可能也越大.显然,如果扩大散列表的存储空间(增加可能的存储位置),就可与降低其负载因子,减小出现冲突的概率.但负载因子越小,散列表里空闲空间的比例也就越大.因此这里也有得失权衡的问题.

另一方面,无论负载因子的情况怎样,冲突总是可能出现的.特别是在实现供其他人使用的一般散列表时,开发者无法了解实际使用中的情况,在设计时就必须考虑冲突的处理问题.由于散列表的重要性和广泛应用,人们深入研究散列表的设计,提出了一些冲突处理技术.基于这些处理方式,形成了多种不同的散列表实现结构.

总结一下,如果要基于散列技术实现字典,必须解决两个大问题:散列函数的设计,冲突消解机制.
### 8.2.3 散列函数
在设计字典时,首先应该根据实际需要确定数据项集合,选定相应的关键码集合KEY.为了实现散列表字典,还要确定一个存储位置区间INDEX,例如,选择从0开始的一段下标.KEY和INDEX是两个有限集,分别为将要定义的散列函数的参数域(定义域)和值域,是定义散列函数的基础.从本质上说,任何从KEY到INDEX的全函数f都能满足散列函数的基本要求,因此对任何$key\in KEY,都有f(key)\in INDEX$,函数值落在合法下标范围内.但另一方面,散列函数的选择可能影响出现冲突的概率.

在设计散列函数时,有价值的考虑包括:
- 函数应能把关键码映射到值域INDEX中尽可能大的部分.显然,如果扩大函数值的范围(在INDEX内),出现冲突的可能性就会下降.如果某个下标不是散列函数的可能值,这个位置可能就无法用到.应该尽量避免这种情况.
- 不同关键码的散列值在INDEX里均匀分布,有可能减少冲突.当然,实际情况还域真实数据里不同关键值出现的分布有关.如果不知道关键码的实际分布(例如,开发一个字典库模块时的i情况就是这样),就只能考虑均匀分布.
- 函数的计算比较简单.这一要求很显然:使用散列表的本意是希望提高效率,而计算散列函数的开销是各种操作的开销中的一部分.
#### 用于整数关键码的若干散列方法
**数字分析法**:对于给定的关键码集合,分析所有关键码中各位数字的出现频率,从中选出分布情况较好的若干数字作为散列函数的值.

__折叠法__:将较长的关键码切分为几段,通过某种运算将它们合并.

**中平方法**:先求出关键码的平方,然后取出中间的几位作为散列值.

从上述实例可以看出,对于整数关键码,散列函数的设计有两方面的追求:其一是把较长的关键码映射到较小的区间,另一个就是尽可能消除关键码与映射值之间明显的规律性.通俗地说,`散列函数地映射关系越乱越好,越不清晰越好`.
#### 常用散列函数
- 除余法,适用于整数关键码:关键码key是整数,用key除以某个不大于散列表长度m的整数p,以得到的余数(或者余数加l,由下标开始值确定)作为散列地址.
- 基数转换法,适用于整数或字符串关键码:先考虑整数关键码.取一个正整数r,把关键码看作基数为r的数(r进制的数),将其转换为十进制或二进制数.通常r取素数以减少规律性.对于其他非整数的关键码,常见做法是先设计一种方法把它转换到整数,而后再用整数散列的方法.在各种散列方法最后,可以用除余法把关键码归入所需范围.

In [1]:
# 一个字符串散列函数:
def str_hash(s):
    h1 = 0
    for c in s:
        h1 = h1 * 29 + ord(c)
    return h1

### 8.3.3. 冲突的内消解:开地址技术
采用散列技术实现字典,冲突是必然出现的事件,因为散列函数是从大集合到小集合的全函数,必然会出现两个不同元素的函数值相同的情况.因此,在设计散列表时,必须确定一种冲突消解方案.

冲突消解方案,从实现方式上可以分为两类:
- `内消解方案`(在基本的存储区内部解决冲突问题)
- `外消解方案`(在基本的存储之外解决冲突)

在散列表的使用中需要插入数据项时,用散列函数根据关键码算出了存储位置,但却发现那里已经有关键码不同的项,这时就知道出现了冲突,必须处理.

对于冲突处理技术,有两方面的基本要求:
1. 保证当前这次存入数据项的工作能正常完成.
2. 保证字典的基本存储性质:在任何时候,从任何以前存入字典而后没有删除的关键码出发,都能找到相应的数据项.

几种常用的冲突消解方法
#### 开地址法和探查序列
内消解的基本方法称为`开地址法`,其基本想法是:在准备插入数据并发现冲突时,设法在基本存储区(顺序表)里为需要插入的数据项另行安排一个位置.为此需要设计一种系统的且易于计算的位置安排方式,称为`探查方式`.

抽象的方法是为散列表定义一种易于计算的探查位置序列.首先定义:$$D=d_0,d_1,d_2,...,$$这里的D是一个整数的递增序列,$d_0=0$.而后定义探查序列为$$H_i=(h(key)+d_i)mod p$$这里的p为一个不超过表长度的数.在实际插入数据项时,如果h(key)位置空闲就直接存入(这相当于使用$d_0$);否则就逐个试探一个个位置$H_i$,直至找到第一个空位时把数据项存入.具体的增量序列有许多可能的设计,例如:
1. 取$D=0,1,2,3,4...$,简单的整数序列,这种方法称为`线性探查`.
2. 设计另一个散列函数$h_2$,令$d_i=i\times h_2(key)$,称为`双散列探查`.
$$MOD(n,d)=n-d*INT(n/d)$$

在双散列探查的过程中,检查的位置以不同方式跳跃进行.这种情况有可能减少关键码堆积的发生.当然,随着表中元素增加,冲突越来越严重的情况不会改变.
#### 检索和删除
这两个操作共同的第一步是找到关键码的位置,或者确定其不存在,也就是检索.

**检索操作**:在开地址法的散列表上做检索,工作过程与插入操作的第一步类似.对于给定的key:
- 调用散列函数,求出key对应的散列地址
- 检查相应存储位置,如果该位置没有数据项,就说明这个散列表里不存在相应的关键码,检索操作以失败结束
- 否则(所检查的位置有数据),比较key与保存在所确定位置的关键码,如果两者匹配则检索以成功结束
- 否则,根据散列表的探查序列找到下一个地址,并回到步骤2.

可见,为判定找不到元素,还需要为单元的`无值状态`确定一种表示方式.

删除操作的第一步也是基于关键码找到要删除的元素,与检索操作的过程完全一样.但开地址法给删除操作带来了一个麻烦:被删除的数据有可能处于其他元素的搜索路径上.如果简单地将其删掉,就可能切断其他元素的探索路径,导致那些元素"失联",此后它们虽然还在字典里,但却不能被找到.这是不允许的.

解决这个问题的方法是:不在被删除的元素位置放入空位标志,而是存入另一个特殊标记.在执行检索操作时,将这种标记看作有元素并继续向下探查;而执行插入操作时,则把这种标记看作空位,把新元素存入这里.
### 8.3.4 外消解技术
在散列表存储区内部解决冲突,可用的手段有限.现在考虑外消解技术.
#### 溢出区方法
另外设置一个溢出存储区.当插入关键码的散列位置没有数据时就直接插入,发生冲突时将相应数据和关键码一起存入溢出区.数据在溢出区里顺序排列.对应的检索和删除操作也是先找到散列位置,如果那里有数据但关键码不匹配,就转到溢出区顺序检索,直至找到要找的关键码,或确定相应数据不存在.

如果冲突项很少,溢出区里的实际数据非常少,这种方式的效果不错.当随着溢出区中数据的增长,字典的性能将趋向线性.

#### 桶散列
另一种可能做法是数据项不存放在散列表的基本存储区里,而是另外存放.在散列表里保存对数据项的引用.基于这种想法可用开发出很多不同的设计.这中设计被称为`桶散列`,下面讨论其中最简单的设计,称为`拉链法`.

在桶散列技术里,散列表的每个元素只是一个引用域,引用着一个保存实际数据的存储桶,`桶散列`的名称由此而来.具体字典可用采用不同的存储桶结构,在拉链法中一个存储桶就i是一个链接的结点表.字典中的数据项并不保存在散列表的主存储区,而是存入相应结点表中的结点里,具有相同散列值的数据项(互为同义词)都保持在这个散列值对应的链表里.采用这种技术,所有数据项都可以统一处理(无论其是否为冲突项),而且允许任意的负载因子.
### 8.3.5 散列表的性质
#### 扩大存储区,用空间交换时间
存储区扩大后,需要相应调整散列函数,以便尽可能利用增加的存储单元(除余法可用在这里)
#### 负载因子和操作效率
散列表的负载因子对效率有决定性的影响.采用内部消解技术时,负载因子$\alpha\le0.7-0.75$时,散列表的平均检索长度接近于整数.

如果采用桶散列技术,负载因子$\alpha$就是桶的平均大小(采用拉链法时是链的平均长度).这种技术可以容忍任意大的负载因子,但随着负载因子变大,检索时间也趋于线性(显然,平均桶长=数据项数/桶数),只是除以一个常量因子.

应该看到,散列表字典的许多性质都是概率性的,而不是确定性.人们说散列表是一种高效的字典实现技术,有些基本假设:
- 实际存入字典的数据(的关键码)的散列函数值分布均匀.
- 字典散列值的负载因子不太高(实验证明应在0.7以下)

在这些假设下,散列表字典检索,插入,删除操作的时间开销可用看作常量.

虽然散列表字典从概率上看极为高效,但事情也有另一方面,有一些必然的情况:
- 其常量时间的操作代价是平均代价,不是每次操作的实际代价.由于不同元素的探查序列长度不一,可能出现有些操作代价较高的情况.
- 一般而言,关键码冲突是必然会发生的情况,并因此导致不同操作的代价只简单额差异可能很大,而且不能事先确知.
- 不断插入元素将导致负载因子不断增大.为了保证操作效率就需要扩大存储,从而导致一次恨到代价的插入操作.而且,无法预知这种高代价操作在什么时候出现.
- 字典内部数据项的排列顺序无法预知,也没有任何保证.

有一些不好的情况也有可能出现,例如:
- 有可能出现实际数据(的关键码)的散列值相对集中的情况,导致一些操作的性能非常差.极端情况是大量常用数据项几种在一个或几个散列值,导致非常长的探查序列,从而使散列表字典非常低效.
- 基于内部消解机制的字典,在长期使用中性能通常会变差.因为在长期使用可能产生很长的已删除元素序列,影响很多操作的效率.
#### 可能技术和实用情况
前面介绍的是基本散列表操作.实际实现还可以考虑许多问题,例如:
- 给用户提供检查负载因子和主动扩大散列表存储区的操作.这样,用户就可与在一段效率要求高的计算之前,根据需要首先设定足够大的存储.
- 对于开地址散列表,记录或检查被删除项的量或比例,在一定情况下自动调整.最简单的方法是另外分配一块存储区,把散列表里的有效数据项重新散列到新区.这种重新散列可用消去开地址散列表里所有已删除项的空位.
## 8.4 集合
集合是数学中最重要的一个基本概念,主要关注个体与个体的汇集之间的关系.其中的个体称为元素,是不加定义的概念,个体的汇集称为集合.

在计算机科学技术领域,具体的数据项可用看作个体,数据项的汇集就是集合,因此可用借用数学的概念和相关定义,处理数据与数据汇集之间的关系.这就是讨论作为数据结构的集合概念和结构时希望考虑的问题.
### 8.4.1 集合的概念,运算和抽象数据类型
#### 概念和集合描述
一个集合就是一些个体的汇集.包含(当前考虑的)所有个体的集合成为全集.

描述集合的一种方法是明确列出其中的所有元素,这种写法称为集合的外延表示,具体写法是用一对大括号.显然,这种外延表示只能描述有穷集合.

集合的另一种描述方法是给出集合中的元素应该满足的性质,这种表示称为集合的描述式,或者集合的内涵表示.

一个集合中元素的个数称为该集合的`基数`,或者说是该集合的`大小`.
#### 抽象数据类型
|ADT Set:|# 集合抽象数据类型|
|---|---|
|Set(self)|# 集合构造函数,创建新的空集|
|is_empty(self)|# 检查self是否为一个空集|
|member(self,elem)|# 检查elem是否为本集合中的元素|
|insert(self,elem)|# 将元素elem加入集合,为变动操作|
|delete(self,elem)|# 从集合中删除元素elem,为变动操作|
|intersection(self,oset)|# 求出本集合和另一集合oset的交集|
|union(self,oset)|# 求出本集合和另一集合oset的并集|
|different(self,oset)|# 求出本集合减去另一集合oset的差集|
|subset(self,oset)|# 判断本集合是否为oset的子集|
|...||
### 8.4.2 集合的实现
任何字典实现技术都可以用于实现集合,为此,只需要把集合元素直接存储在保存字典项(关联)的位置.
#### 简单线性表实现
两套插入和删除方式都可以正确实现集合的功能.
- 插入元素时,检查它是否已在集合里,保证集合(线性表)中元素的唯一性;删除元素时找到第一个相同元素将其删除.
- 插入元素时简单将其加入集合(线性表)里;删除时检查整个表,删除指定元素的所有拷贝(由于插入操作的实现方式,同一元素可能有多个出现).

用简单线性表实现集合,技术比较简单,主要缺点是元素判断和几个集合运算的操作效率都比较低.
#### 排序顺序表实现
#### 散列表实现
考虑基于散列表实现集合,这样:
- 一个集合就是一个散列表
- 插入/删除元素对应于散列表中插入/删除关键码
- 集合元素判断对应于关键码检索
- 各种集合运算都基于上面几个散列表操作,采用建立新散列表的方式实现,没有实质性的困难.
### 8.4.3 特殊实现技术:位向量实现
一个元素是否属于一个集合,是一种二值判断.`集合的位向量表示`.如果在程序里需要使用的一批集合对象有一个(不太大的)公共超集,也就是所,需要实现和使用的集合都是某个超集的子集,就可以考虑位向量技术实现这些集合.用一个n位的二进制序列表示,存在的下标取1否则取0.
## 8.5 Python的标准字典类dict和set
## 8.6 二叉排序树和字典